In [1]:
import pandas as pd
import numpy as np
import requests
import os
import json

In [3]:
FPL_API_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"
PREDICTIONS_FILE = "predictions/all_positions_points-no-availability.csv"
OUTPUT_FILE = "predictions/all_positions_points.csv" 

def get_player_availability():
    """
    Retrieves players avaiablity from fpl api
    
    Returns:
        dict: Dictionary {player_id: availability}
    """
    response = requests.get(FPL_API_URL)
    data = response.json()
    
    availability_dict = {}
    
    for player in data["elements"]:
        player_id = player["id"]
        availability = player["chance_of_playing_this_round"]
        availability_dict[player_id] = availability if availability is not None else 100

    
    return availability_dict

def update_predictions_with_availability(predictions_path, output_path):
    """
    Updates player predictions, multiplying them availability / 100.

    Args:
        predictions_path (str): Ścieżka do pliku z predykcjami (bez availability).
        output_path (str): Ścieżka do pliku wyjściowego.
    """
    availability_dict = get_player_availability()
   
    predictions = pd.read_csv(predictions_path)

    point_columns = [col for col in predictions.columns if col.endswith("_Pts")]
    
    for col in point_columns:
        predictions[col] = predictions.apply(lambda row: row[col] * (availability_dict.get(row["ID"], 0) / 100), axis=1)

    min_points_col = min(point_columns, key=lambda col: predictions[col].min())
    predictions = predictions.sort_values(by=min_points_col, ascending=False)

    predictions.to_csv(output_path, index=False)
    print(f"Updated predictions saved to: {output_path}")

update_predictions_with_availability(PREDICTIONS_FILE, OUTPUT_FILE)

Updated predictions saved to: predictions/all_positions_points.csv
